# Embeddings

Em uma postagem anterior sobre [tokens](https://maximofn.com/tokens/), já vimos a representação mínima de cada palavra. O que corresponde a atribuir um número à divisão mínima de cada palavra.

Entretanto, os transformadores e, portanto, os LLMs, não representam as informações das palavras dessa forma, mas o fazem por meio de "embeddings".

Este caderno foi traduzido automaticamente para torná-lo acessível a mais pessoas, por favor me avise se você vir algum erro de digitação..

Primeiramente, examinaremos duas maneiras de representar palavras dentro de transformadores: `codificação ordinal` e `codificação de um hot`. E, analisando os problemas desses dois tipos de representações, poderemos chegar às "incorporações".

## Codificação ordinal

Essa é a maneira mais básica de representar as palavras dentro dos transformadores. Ela consiste em atribuir um número a cada palavra ou manter os números já atribuídos aos tokens.

No entanto, esse tipo de representação tem dois problemas

 * Imaginemos que table corresponda ao token 3, cat ao token 1 e dog ao token 2. Poderíamos supor que `table = cat + dog`, mas esse não é o caso. Não existe essa relação entre essas palavras. Poderíamos até pensar que, atribuindo os tokens corretos, essa relação poderia ocorrer. No entanto, essa ideia não funciona com palavras que têm mais de um significado, como a palavra "banco".

 * O segundo problema é que as redes neurais fazem internamente muitos cálculos numéricos, portanto, pode ser que mesa tenha o token 3 e seja internamente mais importante do que a palavra cat que tem o token 1.

Portanto, esse tipo de representação de palavras pode ser descartado muito rapidamente.

## Uma codificação quente

O que você faz aqui é usar vetores de dimensão `N`. Por exemplo, vimos que o OpenAI tem um vocabulário de `100277` tokens distintos. Portanto, se usarmos "one hot encoding", cada palavra será representada por um vetor de "100277" dimensões.

No entanto, uma codificação quente tem dois outros problemas importantes.

 * Ele não leva em conta a relação entre as palavras. Portanto, se tivermos duas palavras que sejam sinônimas, por exemplo, `cat` e `feline`, teremos dois vetores diferentes para representá-las.
 No idioma, a relação entre as palavras é muito importante, e não levar essa relação em conta é um grande problema.

 * O segundo problema é que os vetores são muito grandes. Se tivermos um vocabulário de `100277` tokens, cada palavra será representada por um vetor de `100277` dimensões. Isso torna os vetores muito grandes e os cálculos muito caros. Além disso, esses vetores serão todos zeros, exceto na posição correspondente ao token da palavra. Portanto, a maioria dos cálculos serão multiplicações por zero, que são cálculos que não somam nada. Portanto, teremos muita memória alocada para vetores em que só há um 1 em uma determinada posição.

## Embeddings de palavras

Os word embeddings tentam resolver os problemas dos dois tipos anteriores de representações. Para isso, são usados vetores de `N` dimensões, mas, nesse caso, não são usados vetores de 100277 dimensões, mas sim vetores de muito menos dimensões. Por exemplo, veremos que o OpenAI usa `1536` dimensões.

Cada uma das dimensões desses vetores representa uma característica da palavra. Por exemplo, uma dimensão pode representar se a palavra é um verbo ou um substantivo. Outra dimensão pode representar se a palavra é um animal ou não. Outra dimensão pode representar se a palavra é um substantivo próprio ou não. E assim por diante.

No entanto, esses recursos não são definidos manualmente, mas aprendidos automaticamente. Durante o treinamento dos transformadores, os valores de cada uma das dimensões dos vetores são ajustados, de modo que as características de cada uma das palavras sejam aprendidas.

Ao fazer com que cada uma das dimensões da palavra represente uma característica da palavra, as palavras que têm características semelhantes terão vetores semelhantes. Por exemplo, as palavras `cat` e `feline` terão vetores muito semelhantes, pois ambas são animais. E as palavras `table` e `chair` terão vetores semelhantes, pois ambas são móveis.

Na imagem a seguir, podemos ver uma representação tridimensional das palavras e podemos ver que todas as palavras relacionadas a `school` estão próximas, todas as palavras relacionadas a `food` estão próximas e todas as palavras relacionadas a `ball` estão próximas.

![word_embedding_3_dimmension](http://maximofn.com/wp-content/uploads/2023/12/word_embedding_3_dimmension.webp)

Como cada uma das dimensões dos vetores representa uma característica da palavra, podemos realizar operações com palavras. Por exemplo, se subtrairmos a palavra "king" (rei) da palavra "man" (homem) e adicionarmos a palavra "woman" (mulher), obteremos uma palavra muito semelhante à palavra "queen" (rainha). Verificaremos isso mais tarde com um exemplo

### Similaridade entre palavras

Como cada palavra é representada por um vetor N-dimensional, podemos calcular a semelhança entre duas palavras. Para isso, é usada a função de similaridade de cosseno.

Se duas palavras estiverem próximas no espaço vetorial, isso significa que o ângulo entre seus vetores é pequeno, portanto, seu cosseno é próximo de 1. Se houver um ângulo de 90 graus entre os vetores, o cosseno será 0, o que significa que não há semelhança entre as palavras. E se houver um ângulo de 180 graus entre os vetores, o cosseno será -1, ou seja, as palavras são opostas.

![cosine similarity](http://maximofn.com/wp-content/uploads/2023/12/cosine_similarity-scaled.webp)

### Exemplo com embeddings da OpenAI

Agora que já sabemos o que são `embeddings`, vamos dar uma olhada em alguns exemplos com os `embeddings` fornecidos pela `API` `OpenAI`.

Para fazer isso, primeiro precisamos ter o pacote `OpenAI` instalado.

````bash
pip install openai
```

Importamos as bibliotecas necessárias

In [1]:
from openai import OpenAI
import torch
from torch.nn.functional import cosine_similarity

Usamos uma "chave API" da OpenAI. Para fazer isso, vá para a página [OpenAI] (https://openai.com/) e registre-se. Depois de registrado, vá para a seção [API Keys](https://platform.openai.com/api-keys) e crie uma nova `API Key`.

![open ai api key](https://raw.githubusercontent.com/maximofn/alfred/main/gifs/openaix2.gif)

In [2]:
api_key = "Pon aquí tu API key"

Selecionamos o modelo de embeddings que queremos usar. Nesse caso, usaremos o `text-embedding-ada-002`, que é recomendado pela `OpenAI` em sua documentação [embeddings] (https://platform.openai.com/docs/guides/embeddings/).

In [ ]:
model_openai = "text-embedding-ada-002"

Criar um cliente `API

In [ ]:
client_openai = OpenAI(api_key=api_key, organization=None)

Vamos ver como são os `embeddings` da palavra `King`.

In [7]:
word = "Rey"
embedding_openai = torch.Tensor(client_openai.embeddings.create(input=word, model=model_openai).data[0].embedding)

embedding_openai.shape, embedding_openai

(torch.Size([1536]),
 tensor([-0.0103, -0.0005, -0.0189,  ..., -0.0009, -0.0226,  0.0045]))

Como podemos ver, obtemos um vetor de `1536` dimensões.

### Operações com palavras

Vamos obter os embeddings das palavras `king`, `man`, `woman` e `Queen`.

In [19]:
embedding_openai_rey = torch.Tensor(client_openai.embeddings.create(input="rey", model=model_openai).data[0].embedding)
embedding_openai_hombre = torch.Tensor(client_openai.embeddings.create(input="hombre", model=model_openai).data[0].embedding)
embedding_openai_mujer = torch.Tensor(client_openai.embeddings.create(input="mujer", model=model_openai).data[0].embedding)
embedding_openai_reina = torch.Tensor(client_openai.embeddings.create(input="reina", model=model_openai).data[0].embedding)

In [20]:
embedding_openai_reina.shape, embedding_openai_reina

(torch.Size([1536]),
 tensor([-0.0110, -0.0084, -0.0115,  ...,  0.0082, -0.0096, -0.0024]))

Vamos obter a incorporação resultante da subtração da incorporação de "homem" de "rei" e da adição da incorporação de "mulher" a "rei".

In [21]:
embedding_openai = embedding_openai_rey - embedding_openai_hombre + embedding_openai_mujer

In [22]:
embedding_openai.shape, embedding_openai

(torch.Size([1536]),
 tensor([-0.0226, -0.0323,  0.0017,  ...,  0.0014, -0.0290, -0.0188]))

Por fim, comparamos o resultado obtido com a incorporação da `reina`. Para isso, usamos a função `cosine_similarity` fornecida pela biblioteca `pytorch`.

In [23]:
similarity_openai = cosine_similarity(embedding_openai.unsqueeze(0), embedding_openai_reina.unsqueeze(0)).item()

print(f"similarity_openai: {similarity_openai}")

similarity_openai: 0.7564167976379395


Como podemos ver, é um valor muito próximo de 1, portanto, podemos dizer que o resultado obtido é muito semelhante à incorporação da `reina`.

Se usarmos palavras em inglês, obteremos um resultado mais próximo de 1.

In [15]:
embedding_openai_rey = torch.Tensor(client_openai.embeddings.create(input="king", model=model_openai).data[0].embedding)
embedding_openai_hombre = torch.Tensor(client_openai.embeddings.create(input="man", model=model_openai).data[0].embedding)
embedding_openai_mujer = torch.Tensor(client_openai.embeddings.create(input="woman", model=model_openai).data[0].embedding)
embedding_openai_reina = torch.Tensor(client_openai.embeddings.create(input="queen", model=model_openai).data[0].embedding)

In [16]:
embedding_openai = embedding_openai_rey - embedding_openai_hombre + embedding_openai_mujer

In [17]:
similarity_openai = cosine_similarity(embedding_openai.unsqueeze(0), embedding_openai_reina.unsqueeze(0))
print(f"similarity_openai: {similarity_openai}")

similarity_openai: tensor([0.8849])


Isso é normal, pois o modelo OpenAi foi treinado com mais txt em inglês do que em espanhol.